# Simple RAG (Retrieval-Augmented Generation) System for CSV Files

**Tổng quan**

Đoạn code này triển khai một hệ thống Retrieval-Augmented Generation (RAG) cơ bản để xử lý và truy vấn các tài liệu CSV. Hệ thống mã hóa nội dung tài liệu vào một vector store, sau đó có thể được truy vấn để lấy thông tin liên quan.

**Cấu trúc tệp CSV và trường hợp sử dụng**

Tệp CSV chứa dữ liệu khách hàng mẫu, bao gồm các thuộc tính khác nhau như tên, họ, công ty, v.v. Tập dữ liệu này sẽ được sử dụng cho trường hợp sử dụng RAG, tạo điều kiện thuận lợi cho việc xây dựng hệ thống Q&A thông tin khách hàng.

**Các thành phần chính**

* Tải và chia nhỏ các tệp CSV.
* Tạo vector store bằng cách sử dụng FAISS và OpenAI embeddings.
* Thiết lập retriever để truy vấn các tài liệu đã xử lý.
* Tạo câu hỏi và trả lời dựa trên dữ liệu CSV.

**Chi tiết phương pháp**

**Xử lý trước tài liệu**

* Tệp CSV được tải bằng Langchain `Csvloader`.
* Dữ liệu được chia thành các chunk.

**Tạo Vector Store**

* OpenAI embeddings được sử dụng để tạo biểu diễn vector của các chunk văn bản.
* Một vector store FAISS được tạo từ các embeddings này để tìm kiếm độ tương tự hiệu quả.

**Thiết lập Retriever**

* Một retriever được cấu hình để tìm nạp các chunk phù hợp nhất cho một truy vấn nhất định.

**Lợi ích của phương pháp này**

* **Khả năng mở rộng (Scalability):** Có thể xử lý các tài liệu lớn bằng cách xử lý chúng theo từng chunk.
* **Tính linh hoạt (Flexibility):** Dễ dàng điều chỉnh các tham số như kích thước chunk và số lượng kết quả được truy xuất.
* **Hiệu quả (Efficiency):** Sử dụng FAISS để tìm kiếm độ tương tự nhanh chóng trong không gian nhiều chiều.
* **Tích hợp với NLP nâng cao (Integration with Advanced NLP):** Sử dụng OpenAI embeddings để biểu diễn văn bản hiện đại.

**Kết luận**

Hệ thống RAG đơn giản này cung cấp nền tảng vững chắc để xây dựng các hệ thống truy xuất thông tin và trả lời câu hỏi phức tạp hơn. Bằng cách mã hóa nội dung tài liệu vào một vector store có thể tìm kiếm được, nó cho phép truy xuất hiệu quả thông tin liên quan để đáp ứng các truy vấn. Phương pháp này đặc biệt hữu ích cho các ứng dụng yêu cầu truy cập nhanh vào thông tin cụ thể trong tệp CSV.


In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv(dotenv_path='.env')

llm = ChatOpenAI(model="gpt-3.5-turbo")

## CSV File Structure and Use Case

In [2]:
import pandas as pd

file_path = 'data/customers-100.csv'
data = pd.read_csv(file_path)
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


In [6]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [7]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))
vector_store = FAISS(
    embedding_function=OpenAIEmbeddings(),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

vector_store.add_documents(documents=docs)

['88bd41df-8d70-492b-9d70-6332bcc7f93a',
 '69cd15a7-8889-45eb-8f96-90c625a66456',
 'fdab8e6e-1d2f-4b5b-8f72-c273fb2bed2b',
 '0cf8298c-6d78-4eba-a06c-f46e7fce2a77',
 '9d8548fd-acbb-4dfb-b9ca-9b6e6f1372fc',
 '5c9021f1-c536-44aa-a70d-3b89904f7073',
 'c17896bc-938a-4f4f-8e01-9a61b5822985',
 '2e878e8d-6744-49f2-a74d-4edb9adca684',
 '516e6edb-abc9-4a82-bde2-56988fb7bb4d',
 'cac090a3-77a3-4080-b55f-91804de8e241',
 '9c0da227-4df0-4bf1-8a90-939dd7b201b9',
 'd3ab150e-e56c-4bb9-9cc8-e547ef527b24',
 '10f498d6-97de-4765-a035-5492e664d6d2',
 'ee6cc7d1-8dbb-4e48-abed-8ec9f564d57c',
 'c2bd01a4-dd91-4ac6-8cf3-38942bd28bdc',
 'bbf320af-59b8-4277-9932-9ccd762a1c03',
 'a5d93acb-6ea3-4f05-a586-6d2d39e933be',
 'cce08fb3-95a9-494b-ab2d-0d420d7a4b89',
 'fa0a338c-0a85-4b8f-82a5-c9eb173edd7d',
 'f399446d-6902-4aaf-a1a5-1ce2de372ddb',
 '81f0addd-205b-4e27-9091-93bfb30c9817',
 'e807618f-e433-451e-bf00-d21017180078',
 '9c35835e-5f5b-4bd4-bcc4-cd40ab1458cc',
 'c9eddbe3-2faf-452b-bc39-f1e4b1c899b4',
 '4ff3477e-fee0-

In [9]:
# Create the retrieval chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [10]:

answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

'Sheryl Baxter works for the Rasmussen Group.'